Bora lá, vamos tentar filtrar notícias verdadeiras e notícias falsas

Primeiro, precisamos improtar todas as bibliotecas que serão usadas ao longo deste notebook

In [2]:
# -*- coding: utf-8 -*-

import numpy as np # biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd # biblioteca usada para trabalhar com dataframes e análise de dados
import sklearn as skl # sckit-learn
import time # biblioteca usada para calcular o tempo de execucao
import re # biblioteca para expressoes regulares
import os # biblioteca usada para realizar tarefas específicas ao SO


# importa alguns pacotes do sckit-learn
from sklearn import model_selection 
from sklearn import naive_bayes # necessario para usar o metodo naive Bayes
from sklearn import model_selection # necessario para fazer validacao cruzada
from sklearn import metrics # necessario para obter o desempenho da classificacao

import nltk # biblioteca para Processamento de Linguagem Natural
from nltk.stem.porter import PorterStemmer # para fazer a estemização em documentos da lingua inglesa
from nltk.stem import RSLPStemmer # para fazer a estemização em documentos da lingua portuguesa

import unicodedata # sera usada para remover acentos dos documentos em lingua portuguesa

# bibliotecas para geração de gráficos
import matplotlib.pyplot as plt

# biblioteca para tratar com caminho de arquivo
from pathlib import Path

# biblioteca para leitura de csv
import csv

print('Bibliotecas carregadas com sucesso')

Bibliotecas carregadas com sucesso


Separando o download de todos os recursos da biblioteca NLTK, que esse demora

In [ ]:
nltk.download('all')

Separando dados entre dados e treino

In [4]:
def ImportDatasetToArray(pathDataset):
    dataset = []
    target = []
    
    with open(pathDataset, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader, None)
        
        classification = Path(pathDataset).stem
        
        for row in reader:
            dataset.append(row)
            target.append(classification)
            
    return dataset, target

In [5]:
datasetTrue = []
targetTrue = []
datasetTrue, targetTrue = ImportDatasetToArray('./input/data/true.csv')

datasetFake = []
targetFake = []
datasetFake, targetFake = ImportDatasetToArray('./input/data/fake.csv')

dataset = np.asarray(datasetTrue + datasetFake)
target = np.asarray(targetTrue + targetFake)
classes = np.unique(target)

Separando dados entre dados de treino e teste (Pesquisar divisão de treino por meio de cruzada K-fold, supostamente é mais robusta)

In [6]:
porcentagemTreino = 0.8
seed = 20
permutedIndexes = np.random.RandomState(seed).permutation(dataset.shape[0])

datasetShuffled, targetShuffled = dataset[permutedIndexes], target[permutedIndexes]

qtdTreino = int(datasetShuffled.shape[0]*porcentagemTreino)
setTreino, targetTreino = datasetShuffled[0:qtdTreino], targetShuffled[0:qtdTreino]
setTeste, targetTeste = datasetShuffled[qtdTreino:], targetShuffled[qtdTreino:]

In [7]:
print('Qtd. dados de treinamento: %d (%1.2f%%)' %(setTreino.shape[0], (setTreino.shape[0]/dataset.shape[0])*100))
print('Qtd. de dados de teste: %d (%1.2f%%)' %(setTeste.shape[0], (setTeste.shape[0]/dataset.shape[0])*100))
print('Proporção classes treino: %1.2f%%/%1.2f%% (true/fake)' %((targetTreino[targetTreino == 'true'].shape[0]/targetTreino.shape[0]) * 100, (targetTreino[targetTreino == 'fake'].shape[0]/targetTreino.shape[0])*100))
print('Proporção classes testes: %1.2f%%/%1.2f%% (true/fake)' %((targetTeste[targetTeste == 'true'].shape[0]/targetTeste.shape[0])*100, (targetTeste[targetTeste == 'fake'].shape[0]/targetTeste.shape[0]) * 100))

Qtd. dados de treinamento: 2608 (80.00%)
Qtd. de dados de teste: 652 (20.00%)
Proporção classes treino: 50.42%/49.58% (true/fake)
Proporção classes testes: 48.31%/51.69% (true/fake)


Realizando pré-processamento das amostras

Pré-processamentos realizados:
  - Deixar todas as palavras com letras minúsculas
  - Substituir os números pela tag *\<NUMBER\>*
  - Substituir as URLS pela tag *\<URL\>*
  - Substituir os emails pela tag *\<EMAIL\>*
  - Substituir os símbolos de moeda pela tag *\<MONEY\>*
  - Substituir todos os caracteres não-alfanuméricos por um espaço em branco
  - Removido stopwords
  - Aplicado estemização
  - Removido palavras com apenas um caracter

In [8]:
def preprocessing(text, stemming = False, stopwords = False):
    """
    Funcao usada para fazer o tratamento de textos da lingua portuguesa
    
    Parametros: 
        text: variavel do tipo string que contem o texto que devera ser tratado
        
        stemming: variavel do tipo booleana que indica se a estemizacao deve ser aplicada ou nao
        
        stopwords: variavel do tipo booleana que indica se as stopwords devem ser removidas ou nao
    """
    
    # Lower case
    text = text.lower()
    
    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text) 
    
    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)

    #normaliza o símbolo de dólar
    regex = re.compile('[$]+')
    text = re.sub(regex, "<MONEY>", text)    
    
    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')  
    text = re.sub(regex, " ", text)
    
    # normaliza os numeros 
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "<NUMBER>", text)
    
    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('portuguese')]
        text = " ".join( words )
    
    # aplica estemização
    if stemming: 
        stemmer_method = RSLPStemmer()  
        words = text.split() # separa o texto em palavras
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )
    
    # remove palavras de apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )
    
    return text

In [9]:
for i, msg in enumerate(setTreino):
    setTreino[i][1] = preprocessing(setTreino[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(setTreino)-1:
        print(' %d de %d' %(i+1, len(setTreino)), end='\r')
        
for i, msg in enumerate(setTeste):
    setTeste[i][1] = preprocessing(setTeste[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(setTeste)-1:
        print(' %d de %d' %(i+1, len(setTeste)), end='\r')

### Gerando word embeddings

Transformar o texto em um vetor de atributos com valores numéricos. 

In [ ]:
msgTreinoEmb = []
for row in setTreino:
    msgTreinoEmb.append(row[1].split())
    
print('Primeira mensagem de texto')
print(msgTreinoEmb[0])

Treinando o modelo de embeddings

In [ ]:
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=msgTreinoEmb, vector_size=200, window=3, min_count=2, workers=4)

vocabSize = len(model.wv)
print('Tamanho do vocabulário do modelo: ', vocabSize)

Avaliando os temor mais próximos de um determinado termo

In [ ]:
sims = model.wv.most_similar('vacin')
for sim in sims:
    print(sim)

In [13]:
def getDocvector(model, doc): 
    """
    obtem o vetor de cada palavra de um documento e calcula um vetor medio
    """
    
    vectorList = []
    for word in doc:
        try:
            vectorList.append( model.wv[word] )
        except:
            pass # essa termo do python significa que não há nada a fazer
            
    # se a lista de vetores for vazia retorna zero, senao retorna a media dos vetores
    if len(vectorList)>0:
        vetorMedio  = np.mean(vectorList, axis=0)
    else:
        vetorMedio = np.zeros( model.wv.vector_size )
        
    
    return vetorMedio

In [ ]:
targetTreino

In [ ]:
rowTreinoEmb = []
for row in setTreino:
    rowTreinoEmb.append([ getDocvector(model, row[1].split()), row[2], row[3], row[4]])
    
rowTesteEmb = []
for row in setTeste:
    rowTesteEmb.append([ getDocvector(model, row[1].split()), row[2], row[3], row[4]])
    
rowTreinoEmb = np.array(rowTreinoEmb, dtype=object)
rowTesteEmb = np.array(rowTesteEmb, dtype=object)

print("Dimensao de rowTreinoEmb: ", rowTreinoEmb.shape)
print("Dimensao de rowTesteEmb: ", rowTesteEmb.shape)

In [ ]:
rowTreinoEmb = []
for row in setTreino:
    rowTreinoEmb.append(getDocvector(model, row[1].split()))
    
rowTesteEmb = []
for row in setTeste:
    rowTesteEmb.append(getDocvector(model, row[1].split()))
    
rowTreinoEmb = np.array(rowTreinoEmb, dtype=object)
rowTesteEmb = np.array(rowTesteEmb, dtype=object)

print("Dimensao de rowTreinoEmb: ", rowTreinoEmb.shape)
print("Dimensao de rowTesteEmb: ", rowTesteEmb.shape)

In [ ]:
rowTreinoEmb = []
for row in setTreino:
    rowTreinoEmb.append(np.concatenate((getDocvector(model, row[1].split()), np.array([(int(row[2])-16)/15, (int(row[3])-6.5)/5.5, (int(row[4]) - 2020.5)/0.5], dtype='float32'))))
    
rowTesteEmb = []
for row in setTeste:
    rowTesteEmb.append(np.concatenate((getDocvector(model, row[1].split()), np.array([(int(row[2])-16)/15, (int(row[3])-6.5)/5.5, (int(row[4]) - 2020.5)/0.5], dtype='float32'))))
    
rowTreinoEmb = np.array(rowTreinoEmb, dtype=object)
rowTesteEmb = np.array(rowTesteEmb, dtype=object)

print("Dimensao de rowTreinoEmb: ", rowTreinoEmb.shape)
print("Dimensao de rowTesteEmb: ", rowTesteEmb.shape)

In [ ]:
rowTreinoEmb = []
for row in setTreino:
    rowTreinoEmb.append(np.concatenate((getDocvector(model, row[1].split()), np.array([(int(row[2])-1)/30, (int(row[3])-1)/11, int(row[4]) - 2020], dtype='float32'))))
    
rowTesteEmb = []
for row in setTeste:
    rowTesteEmb.append(np.concatenate((getDocvector(model, row[1].split()), np.array([(int(row[2])-1)/30, (int(row[3])-1)/11, int(row[4]) - 2020], dtype='float32'))))
    
rowTreinoEmb = np.array(rowTreinoEmb, dtype=object)
rowTesteEmb = np.array(rowTesteEmb, dtype=object)

print("Dimensao de rowTreinoEmb: ", rowTreinoEmb.shape)
print("Dimensao de rowTesteEmb: ", rowTesteEmb.shape)

Utilizando método Perceptron

In [ ]:
import sklearn
from sklearn.linear_model import Perceptron

# criando um novo perceptron
perceptron = Perceptron(random_state=0, max_iter=30)

# treina o classificador com os dados de treinameto
perceptron.fit(rowTreinoEmb, targetTreino) 
 
# classifica os dados de teste
predicao = perceptron.predict(rowTesteEmb) 
        
print('\nPredição obtida para as 20 primeiras amostras de teste:\n', predicao[0:20])

In [ ]:
resultados = sklearn.metrics.classification_report(targetTeste, predicao, target_names=classes)

print(resultados)